In [16]:
#descargar archivos desde url y almacenarlos en carpeta local
import urllib.request as request
import csv
r = request.urlopen('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
#para imprimir el archivo
''' 
for line in reader:
    print(line)    
'''

' \nfor line in reader:\n    print(line)    \n'

In [17]:
#otra manera e importar los archgivos desde internet
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv',index_col=0)
df.to_csv('truck_event_text_partition.csv')

In [18]:
import os

for root, dirs, files in os.walk('/TEMP/'):
    for file in files:
        if file.endswith('.ipynb'):
            print(os.path.join(root, file))


In [19]:
#prueba de commit en github con el archivo de jupyter lab
filenames = [
    "drivers.csv",
    "timesheet.csv",
    "truck_event_text_partition.csv",
]

In [20]:
#Descargando datos desde repo github
import pandas as pd

url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.csv"
drivers = pd.read_csv(url)
drivers.head()

,driverId,name,ssn,location,certified,wage-plan
0,10,George Vetticaden,621011971,244-4532 Nulla Rd.,N,miles
1,11,Jamie Engesser,262112338,366-4125 Ac Street,N,miles
2,12,Paul Coddin,198041975,Ap #622-957 Risus. Street,Y,hours
3,13,Joe Niemiec,139907145,2071 Hendrerit. Ave,Y,hours
4,14,Adis Cesir,820812209,Ap #810-1228 In St.,Y,hours


In [21]:
import pandas as pd
url2="https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv"
truck_event_text_partition=pd.read_csv(url2,index_col=0) #super importante el index para que no ponga el id del registro
truck_event_text_partition.head()

,truckId,eventTime,eventType,longitude,latitude,eventKey,CorrelationId,driverName,routeId,routeName,eventDate
driverId,,,,,,,,,,,
14,25,59:21.4,Normal,-94.58,37.03,14|25|9223370572464814373,3.660000e+18,Adis Cesir,160405074,Joplin to Kansas City Route 2,2016-05-27-22
18,16,59:21.7,Normal,-89.66,39.78,18|16|9223370572464814089,3.660000e+18,Grant Liu,1565885487,Springfield to KC Via Hanibal,2016-05-27-22
27,105,59:21.7,Normal,-90.21,38.65,27|105|9223370572464814070,3.660000e+18,Mark Lochbihler,1325562373,Springfield to KC Via Columbia Route 2,2016-05-27-22
11,74,59:21.7,Normal,-90.20,38.65,11|74|9223370572464814123,3.660000e+18,Jamie Engesser,1567254452,Saint Louis to Memphis Route2,2016-05-27-22
22,87,59:21.7,Normal,-90.04,35.19,22|87|9223370572464814101,3.660000e+18,Nadeem Asghar,1198242881,Saint Louis to Chicago Route2,2016-05-27-22


<h3>Preparacion </h3>

In [22]:
import sqlite3
conn=sqlite3.connect(':memory:')## aca se indica el nombre de la db
cur= conn.cursor()

Carga de los datos de los eventos de los conductores

In [23]:
#
# executescript() permite enviar varios comandos de SQL
# en la misma cadena de texto.
#

conn.executescript(
    """
DROP TABLE IF EXISTS truck_events;

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING);
"""
)
conn.commit()

In [24]:
#
# La función execute() permite enviar únicamente un comando SQL
# al motor de la base de datos. El siguiente comando es equivalente
# a `SHOW TABLES;` en SQL
#
cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('truck_events',)]

In [25]:
#
# La siguiente sentencia devuelve un string que
# representa los comandos sql para crear la tabla
#
cur.execute(
    "SELECT sql FROM sqlite_master WHERE type='table' and name='truck_events';"
).fetchall()

[('CREATE TABLE truck_events (driverId       INT,\n                           truckId        INT,\n                           eventTime      STRING,\n                           eventType      STRING,\n                           longitude      DOUBLE,\n                           latitude       DOUBLE,\n                           eventKey       STRING,\n                           correlationId  STRING,\n                           driverName     STRING,\n                           routeId        STRING,\n                           routeName      STRING,\n                           eventDate      STRING)',)]

In [26]:
#
# Se imprime el comando sql equivalente para cada
# tabla existente en la base de datos
#
for a in cur.execute("SELECT sql FROM sqlite_master WHERE type='table';").fetchall():
    print(a[0])

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING)


<h4>Carga de datos usando INSERT INTO</h4>
Este método de carga consiste en crear una lista de tupas, donde cada elemento de la tupla es un campo de la tabla.


In [27]:
with open('truck_event_text_partition.csv','rt') as file:
    truck_event=file.readlines()
len(truck_event)
truck_event=[line.replace('\n','') for line in truck_event]
#truck_event=[line[][1:] for line in truck_event]
truck_event[:3]


['driverId,truckId,eventTime,eventType,longitude,latitude,eventKey,CorrelationId,driverName,routeId,routeName,eventDate',
 '14,25,59:21.4,Normal,-94.58,37.03,14|25|9223370572464814373,3.66e+18,Adis Cesir,160405074,Joplin to Kansas City Route 2,2016-05-27-22',
 '18,16,59:21.7,Normal,-89.66,39.78,18|16|9223370572464814089,3.66e+18,Grant Liu,1565885487,Springfield to KC Via Hanibal,2016-05-27-22']

In [28]:
truck_event=[line.replace('\n','') for line in truck_event]
# Separa los campos por comas
truck_event=[line.split(',') for line in truck_event]
#convierte a la fila en una tupla
truck_event=[tuple(line) for line in truck_event]
truck_event[:3]
#descartando la primera que es la cabezera 
truck_event=truck_event[1:]
truck_event[:3]
#NOTA importante: para cargar la lista en la base de datos se debe ingresar como una tubpla

[('14',
  '25',
  '59:21.4',
  'Normal',
  '-94.58',
  '37.03',
  '14|25|9223370572464814373',
  '3.66e+18',
  'Adis Cesir',
  '160405074',
  'Joplin to Kansas City Route 2',
  '2016-05-27-22'),
 ('18',
  '16',
  '59:21.7',
  'Normal',
  '-89.66',
  '39.78',
  '18|16|9223370572464814089',
  '3.66e+18',
  'Grant Liu',
  '1565885487',
  'Springfield to KC Via Hanibal',
  '2016-05-27-22'),
 ('27',
  '105',
  '59:21.7',
  'Normal',
  '-90.21',
  '38.65',
  '27|105|9223370572464814070',
  '3.66e+18',
  'Mark Lochbihler',
  '1325562373',
  'Springfield to KC Via Columbia Route 2',
  '2016-05-27-22')]

In [30]:
## Carga a partir de la lista de tuplas
# contenidas en data
cur.executemany("INSERT INTO truck_events VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", truck_event)
# Verificación
#
cur.execute("SELECT * FROM truck_events LIMIT 1;").fetchall()

<h3> Consulta de datos   </h3>

SELECT
    what
FROM
    tbl_name
[WHERE where-conditions]
[ORDER BY column_name];

In [31]:
cur.execute("SELECT *FROM truck_events LIMIT 3;").fetchall()

[(14,
  25,
  '59:21.4',
  'Normal',
  -94.58,
  37.03,
  '14|25|9223370572464814373',
  3660000000000000000,
  'Adis Cesir',
  160405074,
  'Joplin to Kansas City Route 2',
  '2016-05-27-22'),
 (18,
  16,
  '59:21.7',
  'Normal',
  -89.66,
  39.78,
  '18|16|9223370572464814089',
  3660000000000000000,
  'Grant Liu',
  1565885487,
  'Springfield to KC Via Hanibal',
  '2016-05-27-22'),
 (27,
  105,
  '59:21.7',
  'Normal',
  -90.21,
  38.65,
  '27|105|9223370572464814070',
  3660000000000000000,
  'Mark Lochbihler',
  1325562373,
  'Springfield to KC Via Columbia Route 2',
  '2016-05-27-22')]

<h3>Obtención de un subconjunto de registros</h3>
En SQLite3 es posible obtener un subconjunto de datos y almacenarlo en una nueva tabla a partir de una consulta que permita obtener los datos deseados. En el siguiente código, se crea la tabla truck_events_subset con los primeros 100 registros de la tabla truck_events.


In [35]:
conn.executescript(
    """
    DROP TABLE IF EXISTS truck_events_subset;

    CREATE TABLE truck_events_subset
    AS
    SELECT *
    FROM truck_events 
    LIMIT 100;
    """
)

conn.commit()

In [42]:
#consulta
conn.execute("SELECT *FROM truck_events_subset LIMIT 3;").fetchall()


[(14,
  25,
  '59:21.4',
  'Normal',
  -94.58,
  37.03,
  '14|25|9223370572464814373',
  3660000000000000000,
  'Adis Cesir',
  160405074,
  'Joplin to Kansas City Route 2',
  '2016-05-27-22'),
 (18,
  16,
  '59:21.7',
  'Normal',
  -89.66,
  39.78,
  '18|16|9223370572464814089',
  3660000000000000000,
  'Grant Liu',
  1565885487,
  'Springfield to KC Via Hanibal',
  '2016-05-27-22'),
 (27,
  105,
  '59:21.7',
  'Normal',
  -90.21,
  38.65,
  '27|105|9223370572464814070',
  3660000000000000000,
  'Mark Lochbihler',
  1325562373,
  'Springfield to KC Via Columbia Route 2',
  '2016-05-27-22')]

In [45]:
#obtencion de un subconjunto de columnas
conn.executescript(
    """
    DROP TABLE IF EXISTS specific_columns;

    CREATE TABLE specific_columns
    AS 
    SELECT
        driverId,
        eventTime,
        eventType
    FROM 
        truck_events_subset;

    SELECT * FROM specific_columns LIMIT 1;
"""
)
conn.commit()


In [49]:
#Consulta registros del 2 al 5
cur.execute("SELECT * FROM specific_columns LIMIT 2,3;").fetchall()

[(27, '59:21.7', 'Normal'),
 (11, '59:21.7', 'Normal'),
 (22, '59:21.7', 'Normal')]

In [51]:
#registros para driver id=11

conn.execute("SELECT * FROM specific_columns WHERE driverId==11;").fetchall()

[(11, '59:21.7', 'Normal'),
 (11, '59:22.5', 'Normal'),
 (11, '59:28.3', 'Normal'),
 (11, '59:30.0', 'Normal'),
 (11, '59:31.8', 'Normal')]

In [52]:
#tipos de eventos
cur.execute("SELECT DISTINCT eventType FROM specific_columns;").fetchall()

[('Normal',)]

<h3>Escritura de la tabla en el disco</h3>
Para escribir en un archivo local se tiene que seguir los siguientes pasos

In [55]:
##
# Obtención de los datos como una lista de tuplas
data=cur.execute("SELECT *FROM truck_events_subset;").fetchall()
#conversion de los elementos a strings
text = [[str(element) for element in line ] for line in data ]

#concatenacion de strings
text =[",".join(line) for line in text ]
text = "\n".join(text)

open("data.csv","wt").write(text)


14688